## demo for parsing

In [3]:
%load_ext autoreload
%autoreload 2
from tree_sitter_languages import get_parser
parser = get_parser("java")

with open("test.java", "r") as file:
    source_code = file.read()
print(source_code)

import utils
tree = parser.parse(bytes(source_code, "utf8"))
print(utils.format_code(tree.root_node.sexp()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
import java.io.*;
import java.lang.*;
import java.util.*;
import java.math.*;

class SumSeries {
    /**
     * * Write a function to calculate the sum of the positive integers of
     * n+(n-2)+(n-4)... (until n-x =< 0).
     *
     * > sumSeries(6)
     * 12
     * > sumSeries(10)
     * 30
     * > sumSeries(9)
     * 25
     */
    public static int sumSeries(int n) {
        int sum = 0;
        for (int i = n; i > 0; i = i - 2) {
            sum += i;
        }
        return sum;
    }
}
(program
    (import_declaration
        (scoped_identifier
            scope: (identifier)
            name: (identifier)
        )
        (asterisk)
    )
    (import_declaration
        (scoped_identifier
            scope: (identifier)
            name: (identifier)
        )
        (asterisk)
    )
    (import_declaration
        (scoped_identifier
            scope: (identifier)
            name: (ide

/home/hzc/.conda/envs/ProofT5/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


## Parse all solutions

In [15]:
# get program model from a java file
def getProgram(file_path):
    with open(file_path, "r") as file:
        source_code = file.read()
        utils.source_code_line = source_code.split("\n")
    tree = parser.parse(bytes(source_code, "utf8"))
    return utils.visit_program(tree.root_node)

# print program as a java file
def printJava(program):
    return utils.addIndentation(program.toString())

# validate the parser and the java generation
def validateJava(file_path,isMBJP=True):
    try:
        program = getProgram(file_path)
        javaCode= printJava(program)
        if isMBJP: # validating a mbjp benchmark solution
            from mxeval.execution import check_correctness_java as check
            import json
            with open("mbjp.json", "r") as file:
                mbjp = json.load(file)
            mbjp_name=file_path.split("/")[-1].split(".")[0].replace("_","/") #MBJP/1
            for data in mbjp:
                if data['task_id']==mbjp_name:
                    result=check(data,javaCode,solution_complete=True)
                    if result['passed']:
                        return True
                    else:
                        print(f"\nError in parsing java file: {file_path}")
                        print(javaCode)
                        print(result['result'])
                        return False
            return False
        else:
            parser.parse(bytes(javaCode, "utf8"))
            return True
    except Exception as e:
        print(f"\nError in parsing java file: {file_path}")
        print(e)
        return False
    
print(validateJava("mbjp/MBJP_103.java"))

True


In [16]:
import os
import subprocess
# Path to the solutions folder
folder_path = "mbjp"

# Get the list of Java code files in the folder
java_files = [file for file in os.listdir(folder_path) if file.endswith(".java")]

java_files.sort()
error_cnt=0
# Loop through each Java code file
for file_name in java_files[:100]:
    file_path = os.path.join(folder_path, file_name)
    
    # using the parser to validate the java file
    if not validateJava(file_path):
        error_cnt+=1
print(f"Total errors: {error_cnt} out of {len(java_files)} files.")


Error in parsing java file: mbjp/MBJP_104.java
import java.io.*;
import java.lang.*;
import java.util.*;
import java.math.*;
class SortSublists {
    public static List<List<String>> sortSublists(List<List<String>> inputList) {
        List<List<String>> outList = new ArrayList<>();
        for ( List<String> list : inputList ) {
            if ( list.size() == 0 ) {
                continue;
            }
            Collections.sort(list, new Comparator<String>());
            outList.add(list);
        }
        return outList;
    }
}

/home/hzc/Proof2Prog/translation/mxeval/mxeval/java_exec_eval/qbIFGooFgV/main.java:12: error: Comparator is abstract; cannot be instantiated
            Collections.sort(list, new Comparator<String>());
                                   ^
1 error


Error in parsing java file: mbjp/MBJP_119.java
generic_type中有未知类型scoped_type_identifier Map.Entry

Error in parsing java file: mbjp/MBJP_12.java
import java.io.*;
import java.lang.*;
import java.util.*;


## Translating Program Model into Coq Proof

In [15]:
import shutil
import os
# coq proof test
def validateCoqProof(file_path):
    program=getProgram(file_path)
    coqProof=program.toCoq().toString()
    prefix="""From PLF Require Import Syntax.
Open Scope string_scope.

Example prog_well_typed : exists p, program_well_typed p.
Proof with (simpl;try(reflexivity)).
unfold program_well_typed.
eexists.
eexists."""
    suffix="""  Unshelve.
  all: apply STyVoid.
Defined.
Definition prog := the_exists_term (prog_well_typed).
Print prog."""
    coqProof=f"{prefix}\n{coqProof}\n{suffix}"
    with open("../coq_code/test.v", "w") as file:
        file.write(coqProof)
    res = subprocess.run(
                # coqc -Q ../coq_code PLF ../coq_code/test.v
                ["coqc", "-Q","../coq_code", "PLF", "../coq_code/test.v"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=60,
            )
    if res.returncode != 0:
        def copy_and_rename_test_v(file_path):
            # Get the directory path and file name
            directory = os.path.dirname(file_path)
            file_name = os.path.basename(file_path)
            
            # Create the new file name with the same prefix as the original file
            new_file_name = file_name.split(".")[0] + ".v"
            
            # Create the new file path
            new_file_path = os.path.join("../coq_code", new_file_name)
            
            # Copy the test.v file to the new file path
            shutil.copyfile("../coq_code/test.v", new_file_path)
            shutil.copyfile(file_path, os.path.join("../coq_code", file_name))
            
            return new_file_path

        copy_and_rename_test_v(file_path)
        
        print(f"\nError in parsing coq proof: {file_path}")
        print(res.stderr.decode("utf-8"))
        return False
    else:
        return True

validateOne=1
if validateOne:
    validateCoqProof("mbjp/MBJP_115.java")
else:   
    error_cnt=0
    # Loop through each Java code file
    for file_name in java_files[:]:
        file_path = os.path.join(folder_path, file_name)
        # using the parser to validate the java file
        if not validateJava(file_path):
            error_cnt+=1
            continue

        # using coqc to validate the coq proof
        if not validateCoqProof(file_path):
            error_cnt+=1
    print(f"Total errors: {error_cnt} out of {len(java_files)} files.")


Error in parsing coq proof: mbjp/MBJP_115.java
File "../coq_code/test.v", line 44, characters 48-49:
Error: This proof is focused, but cannot be unfocused this way




In [16]:
def remove_all_mbjp_files_in_coq_folder():
    folder_path = "../coq_code"
    # Get the list of Java code files in the folder
    java_files = [file for file in os.listdir(folder_path) if file.startswith("MBJP")]

    # Loop through each Java code file
    for file_name in java_files:
        file_path = os.path.join(folder_path, file_name)
        os.remove(file_path)
remove_all_mbjp_files_in_coq_folder()